# ＊PYTORCHによるLSTMの実装


- 環境:Ubuntu18.04
- GPU:Quadro RTX 8000
- ドライバー:NVIDIA-SMI 460.32.03, Driver Version: 460.32.03, CUDA Version: 11.2
- ./data:train.tsv, test.tsv,test.csv
- EarlyStoppingを利用する場合はhttps://github.com/Bjarten/early-stopping-pytorch からpytorchtools.pyをutilsにインストールし学習・検証のコメントアウトを外すこと

# ＊事前準備


In [ ]:
!conda create -n pytorch_bert python=3.7
!conda activate pytorch_bert
!conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cudatoolkit=11.1 -c pytorch -c conda-forge
!conda install jupyter
!pip install transformers==4.5.0
!pip install torchtext==0.9.0
!pip install tqdm
!pip install attrdict
!pip install mojimoji
!pip install mecab-python3
!pip install mecabrc
!pip install unidic_lite
!pip install sentencepiece
!pip install fugashi

# ＊ライブラリのインストール

In [ ]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext
import glob
import os
import io
import string
import re
import sys
import random
import mojimoji
import string
import pickle

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel, AutoTokenizer
path_result="./result/"
path_weights="./weights/"
max_length=128

## 前処理

In [ ]:
def preprocessing_text(text):
    # 半角・全角の統一
    text = mojimoji.han_to_zen(text) 
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    #どっちでも
    text = re.sub(',', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]+', '0', text)  # 数字

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        #if (p == "."):
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")
        return text

# ＊TorchtextでDatasetの作成

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 単語分割用のTokenizerを用意
tokenizer_bert = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
#tokenizer_bert = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert):
    text = preprocessing_text(text)
    ret = tokenizer.encode(text, max_length=max_length, truncation=True, return_tensors='pt')[0]
    return ret
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=False, lower=False, include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)

LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False, dtype = torch.float)

# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path="data/", train='train.tsv',
    test='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# ＊Dataloaderの作成

In [ ]:
batch_size = 32  # BERTでは16、32あたりを使用する
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))

train_dl = torchtext.legacy.data.Iterator(train_val_ds, batch_size=batch_size, train=True)
train_dl_val = torchtext.legacy.data.Iterator(train_ds, batch_size=batch_size, train=True)
val_dl = torchtext.legacy.data.Iterator(val_ds, batch_size=batch_size, train=False, sort=False)
test_dl = torchtext.legacy.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict_val = {"train": train_dl_val, "val": val_dl}
dataloaders_dict = {"train": train_dl}

# 動作確認

In [ ]:
# 動作確認 テストデータのデータセットで確認
batch = next(iter(test_dl))
print(batch)
print(batch.Text)
print(batch.Label)

In [ ]:
# ミニバッチの1文目を確認してみる
text_minibatch_1 = (batch.Text[0][1]).numpy()

# IDを単語に戻す
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)

In [ ]:
tokenizer_bert = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
#tokenizer_bert=BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
text=tokenizer_bert("鬼滅の刃は面白いです")
print(text)
text2=tokenizer_bert.convert_ids_to_tokens(text["input_ids"])
print(text2)

# ＊LSTMモデルを構築

In [ ]:
from transformers import BertModel
#model_name="cl-tohoku/bert-base-japanese-whole-word-masking"
model_name="cl-tohoku/bert-base-japanese-v2"
net_bert=BertModel.from_pretrained(model_name)

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self, net_bert, input_size, hidden_size):
        super(LSTMClassifier, self).__init__()

        self.input_size=input_size
        self.hidden_size=hidden_size
        
        # BERTモジュール
        self.bert = net_bert  # BERTモデル
        
        # 前方向と後ろ方向の最後の隠れ層ベクトルを結合したものを受け取るので、hidden_dimを2倍している
        self.bilstm = nn.LSTM(input_size=input_size,  hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.cls = nn.Linear(in_features=hidden_size * 2, out_features=1)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        
        with torch.no_grad():  # 勾配計算なし
        # 単語ベクトルを計算
            # 最終層の隠れ状態ベクトルを取得
            #last_hidden_states = output[0]
            output  = self.bert(input_ids)
            encoded_layers = output['last_hidden_state']
            pooler_output = output['pooler_output'] 
            
        _, output_hc =self.bilstm(encoded_layers)
        output = torch.cat([output_hc[0][0], output_hc[0][1]], dim=1)
        
        out = self.cls(output)

        return out

In [ ]:
# モデル構築
net = LSTMClassifier(net_bert, 768, 768)
# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


# ＊LSTMのファインチューニング

## パターン1：全てのパラメータを更新

In [ ]:
# 最適化手法の設定

optimizer = optim.Adam(net.bilstm.parameters(), lr=1e-3)

# 損失関数の設定
criterion = nn.MSELoss()

## パターン2：BILSTMと識別器のパラメータを更新（BERTは更新しない）

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 最後のBILSTMを勾配計算ありに変更
for name, param in net.bilstm.named_parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for name, param in net.cls.named_parameters():
    param.requires_grad = True

# 最適化手法の設定

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.cls.parameters(), 'lr': 1e-3}
], betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.MSELoss()

# ＊学習・検証

## ＊ 開発データでハイパーパラメータを決定

In [ ]:
# モデルを学習させる関数を作成
#from utils.pytorchtools import EarlyStopping
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    #エポック数,Acuraccy,Loss保存用
    Epochs=[]
    Loss_train=[]
    Loss_val=[]
    
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    #early_stopping = EarlyStopping(patience=10)
    
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size
    n=0
    
    #時間
    start =time.time()
    
    # epochのループ
    for epoch in range(num_epochs):
        """
        if early_stopping.early_stop:
            print("Early Stopping")
            break # 打ち切り
        # epochごとの訓練と検証のループ
        """
        
        for phase in ['train', 'val']:
            count=0
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに
                count=1
            
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # Bertに入力
                    outputs = net(inputs)

                    #loss = torch.sqrt(criterion(outputs, labels)) # 損失を計算
                    #loss = criterion(outputs, labels)
                    loss = criterion(outputs.squeeze(), labels.squeeze())
                    

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        
                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 本イテレーションのRMSE:{}'.format(iteration, loss.item(), duration, loss.item()))
                            t_iter_start = time.time()
                            
                            #early_stopping(loss.item(), net)
                                       
                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    
            # epochごとのloss
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss))
            
            t_epoch_start = time.time()
            
            if count == 0:
                Loss_train.append(epoch_loss)
            elif count==1:
                Loss_val.append(epoch_loss)
                
        Epochs.append(epoch+1)
    t=time.time()
    print("Time:{:.4f}sec".format(t-start))
        
    return net, Epochs, Loss_train, Loss_val


In [ ]:
# 学習・検証を実行する。1epochに20分ほどかかります
num_epochs = 10
net_trained, Epochs, Loss_train, Loss_val= train_model(net, dataloaders_dict_val,
                          criterion, optimizer, num_epochs=num_epochs)


## ＊学習時のEpochsごとのLossを出力

In [ ]:
import matplotlib.pyplot as plt
#plt.axes().set_aspect("equal")
#Loss_test=[]
plt.plot(Epochs, Loss_train,color="blue",label="train")
plt.plot(Epochs, Loss_val,color="red",label="val")
#plt.plot(Epochs,Loss_test,color="red",label="test")
plt.xticks(Epochs)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.savefig(path_result+"Loss_val.png")

## ＊ 全データで学習

In [ ]:
# モデルを学習させる関数を作成
#from utils.pytorchtools import EarlyStopping
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    #エポック数,Acuraccy,Loss保存用
    Epochs=[]
    Loss_train=[]
    Loss_val=[]
    
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    #early_stopping = EarlyStopping(patience=10)
    
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size
    n=0
    
    #時間
    start =time.time()
    
    # epochのループ
    for epoch in range(num_epochs):
        """
        if early_stopping.early_stop:
            print("Early Stopping")
            break # 打ち切り
        # epochごとの訓練と検証のループ
        """
        
        #for phase in ['train', 'val']:
        for phase in ['train']:
            count=0
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに
                count=1
            
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # Bertに入力
                    outputs = net(inputs)

                    #loss = torch.sqrt(criterion(outputs, labels)) # 損失を計算
                    #loss = criterion(outputs, labels)
                    loss = criterion(outputs.squeeze(), labels.squeeze())
                    

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        
                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 本イテレーションのRMSE:{}'.format(iteration, loss.item(), duration, loss.item()))
                            t_iter_start = time.time()
                            
                            #early_stopping(loss.item(), net)
                                       

                    iteration += 1

                    # 損失の合計を更新
                    epoch_loss += loss.item() * batch_size
                    
            # epochごとのloss
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss))
                
            t_epoch_start = time.time()
            
            if count == 0:
                Loss_train.append(epoch_loss)
            elif count==1:
                Loss_val.append(epoch_loss)
        Epochs.append(epoch+1)
        
    t=time.time()
    time_val = t-start
    print("Time:{:.4f}sec".format(time_val))
        
    return net, Epochs, Loss_train, Loss_val, time_val


In [ ]:
# 学習・検証を実行する。1epochに20分ほどかかります
num_epochs = 5
net_trained, Epochs, Loss_train, Loss_val, time_val  = train_model(net, dataloaders_dict_val,
                          criterion, optimizer, num_epochs=num_epochs)

In [ ]:
# 学習したネットワークパラメータを保存します
save_path = path_weights+'bert_fine_tuning.pth'
torch.save(net_trained.state_dict(), save_path)

## ＊＊＊保存しているモデルを使用する場合＊＊＊

In [ ]:
from transformers import BertModel
#model_name="cl-tohoku/bert-base-japanese-whole-word-masking"
model_name="cl-tohoku/bert-base-japanese-v2"
net_bert=BertModel.from_pretrained(model_name)

class LSTMClassifier(nn.Module):

    def __init__(self, net_bert, input_size, hidden_size):
        super(LSTMClassifier, self).__init__()

        self.input_size=input_size
        self.hidden_size=hidden_size
        
        # BERTモジュール
        self.bert = net_bert  # BERTモデル
        
        # 前方向と後ろ方向の最後の隠れ層ベクトルを結合したものを受け取るので、hidden_dimを2倍している
        self.bilstm = nn.LSTM(input_size=input_size,  hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.cls = nn.Linear(in_features=hidden_size * 2, out_features=1)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        
        with torch.no_grad():  # 勾配計算なし
        # 単語ベクトルを計算
            # 最終層の隠れ状態ベクトルを取得
            last_hidden_states = outputs[0]
            output  = self.bert(input_ids)
            encoded_layers = output['last_hidden_state']
            pooler_output = output['pooler_output'] 
            
        _, output_hc =self.bilstm(encoded_layers)
        output = torch.cat([output_hc[0][0], output_hc[0][1]], dim=1)
        
        out = self.cls(output)

        return out

In [ ]:
# モデル構築
net_trained = BertClassifier(net_bert)

# 訓練モードに設定
net_trained.train()

print('ネットワーク設定完了')

#学習済みモデルを読み込む
model_path = './weights/bert_fine_tuning.pth'
net_trained.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## ＊＊＊前処理用（テストデータのみ）＊＊＊

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel, AutoTokenizer
path_result="./result/"
path_weights="./weights/"
max_length=128
batch_size=16

#テストデータのみ
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
# 単語分割用のTokenizerを用意
tokenizer_bert = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
#tokenizer_bert = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

def preprocessing_text(text):
    # 半角・全角の統一
    text = mojimoji.han_to_zen(text) 
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]+', '0', text)  # 数字

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        #if (p == "."):
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")
        return text

# 前処理と単語分割をまとめた関数を定義
# 単語分割の関数を渡すので、tokenizer_bertではなく、tokenizer_bert.tokenizeを渡す点に注意
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)  # tokenizer_bert
    return ret
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
#max_length = 256
#max_length=128
TEXT = torchtext.data.legacy.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True, lower=False, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token="[PAD]",unk_token='[UNK]')

LABEL = torchtext.data.legacy.Field(sequential=False, use_vocab=False, dtype = torch.float)

# フォルダ「data」から各tsvファイルを読み込みます
# BERT用で処理するので、10分弱時間がかかります
train_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path=DATA_PATH, train='train_dumy.tsv',
    test='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

test_dl = torchtext.legacy.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)
# 辞書オブジェクトにまとめる
dataloaders_dict = {"test": test_dl}

## ＊検証

In [ ]:
import pandas as pd
#cc=nn.Softmax(dim=1)
# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

predicted_label=[]#予測ラベル
ture_label = []
count=0

start =time.time()
for batch in tqdm(test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル
    epoch_loss=0.0

    
    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        outputs = net_trained(inputs)

        #loss = torch.sqrt(criterion(outputs, labels)) # 損失を計算
        #loss = criterion(outputs, labels)
        loss = criterion(outputs.squeeze(), labels.squeeze())
        
        for i in range(batch_size):
            try:
                p_label = outputs[i].item()
                t_label = labels[i].item()
                
                predicted_label.append(p_label)
                ture_label.append(t_label)
                count+=1
            except:
                break
        
        
        epoch_loss += loss.item() * batch_size
        
epoch_loss = epoch_loss / len(test_dl.dataset)

t=time.time()
time_test =t-start
print("Time:{:.4f}sec".format(time_test))

print('テストデータ{}個でのRMSE：{}'.format(len(test_dl.dataset), epoch_loss))

df = pd.read_csv("./data/test.csv", names=("TEXT", "LABEL"), engine="python", encoding="utf-8-sig")
df["PREDICT"] = np.nan   #予測列を追加

for index in range(count):
    df.at[index, "PREDICT"] = predicted_label[index]
    
df.to_csv("./result/predicted_test.csv", encoding="utf-8-sig", index=False)


## ＊MSE・RMSEの出力

In [ ]:
from sklearn.metrics import mean_squared_error
mse_loss = mean_squared_error(ture_label, predicted_label, squared=True)
print("MSE：{}".format(mse_loss))

#import numpy as np
#rmse = np.sqrt(mean_squared_error(ture_label, predicted_label, squared=True))
rmse = mean_squared_error(ture_label, predicted_label, squared=False)
print("RMSE：{}".format(rmse))

## ＊MAEの出力

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(ture_label, predicted_label)
print("MAE{}".format(mae))

## ＊R2（決定係数）

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(ture_label, predicted_label)
print(r2)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# yyplot 作成関数
def yyplot(y_label, y_pred):
    yvalues = np.concatenate([y_label, y_pred])
    ymin, ymax, yrange = np.amin(yvalues), np.amax(yvalues), np.ptp(yvalues)
    plt.axes().set_aspect("equal")
    #plt.figure(figsize=(8, 8))
    plt.scatter(y_label, y_pred)
    plt.plot([ymin - yrange * 0.01, ymax + yrange * 0.01], [ymin - yrange * 0.01, ymax + yrange * 0.01])
    plt.xlim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.ylim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.xlabel('Label')
    plt.ylabel('Pred')
    plt.savefig(path_result+"pred.png")
    #plt.title('Observed-Predicted Plot')
    #plt.tick_params(labelsize=1)
    plt.show()
    return plt

# yyplot の実行例
np.random.seed(0)
y_obs = np.random.normal(size=(1000, 1))
y_pred = y_obs + np.random.normal(scale=0.3, size=(1000, 1))
fig = yyplot(ture_label, predicted_label)

In [ ]:
with open("{}RMSE.txt".format(path_result),"a",encoding="utf-8") as f:
    f.write("MSE:{}\n".format(mse_loss))
    f.write("RMSE:{}\n".format(rmse))
    f.write("MAE:{}\n".format(mae))
    f.write("R2:{}\n".format(r2))
    f.write("Time_val:{:.4f}sec\n".format(time_val))
    f.write("Time_test:{:.4f}sec\n".format(time_test))
    f.close()